### Check Dimension Size
Find for each dimension their size/length and check if they are common for each xarray (timeperiod)

In [ ]:
# Get the shapes from the first xarray in the list
#display(cerra)
first_cerra_array = cerra[0]
cerra_lat_shape = first_cerra_array['latitude'].shape
cerra_lon_shape = first_cerra_array['longitude'].shape
cerra_time_steps = first_cerra_array['time'].shape[0]
print(cerra_lat_shape, cerra_lon_shape, cerra_time_steps)
display(first_cerra_array)

tim = []
long = []
lat = []

# Iterate through the list to find common shapes
for xarr in cerra[1:]:
    if xarr['latitude'].shape != cerra_lat_shape: 
        #print("Different latitude/longitude shapes found.")
        if xarr['latitude'].shape not in lat:
            lat.append(xarr['latitude'].shape[0])#, xarr['longitude']])

    if xarr['longitude'].shape != cerra_lon_shape:
        if xarr['longitude'].shape not in long:
            long.append(xarr['longitude'].shape[0])#lat.append([xarr['longitude'].shape[0], xarr['latitude']])

    if xarr['time'].shape[0] != cerra_time_steps:
        #print("Different time steps found.")
        tim.append([xarr['time'].shape[0], xarr['time']])
    

# time is the same as with cerra 
for l in long:
    print(l)

    

(501,) (801,) 248


<xarray.Dataset>
Dimensions:    (longitude: 801, latitude: 501, time: 248)
Coordinates:
  * longitude  (longitude) float32 -10.0 -9.95 -9.9 -9.85 ... 29.9 29.95 30.0
  * latitude   (latitude) float32 60.0 59.95 59.9 59.85 ... 35.1 35.05 35.0
  * time       (time) datetime64[ns] 2019-01-01 ... 2019-01-31T21:00:00
Data variables:
    t2m        (time, latitude, longitude) float32 ...
    orog       (time, latitude, longitude) float32 0.2687 0.3937 ... -0.1063
    lsm        (time, latitude, longitude) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
Attributes:
    Conventions:  CF-1.6
    history:      2023-05-26 08:30:56 GMT by grib_to_netcdf-2.28.0: grib_to_n...

## Check shape among data sets
Check if era5 and cerra data set both share a common shape.

In [ ]:
if cerra_time_steps != era5_time_steps:
    print("time dimension doesn't match")

if cerra_lon_shape != era5_lat_shape:
    print("time dimension doesn't match")
    print("era: ", era5_lat_shape, "cerra: ", cerra_lat_shape)

if cerra_lat_shape != era5_lon_shape:
    print("time dimension doesn't match")
    print("era: ", era5_lon_shape, "cerra: ", cerra_lon_shape)


time dimension doesn't match
era:  (221,) cerra:  (501,)
time dimension doesn't match
era:  (281,) cerra:  (801,)


#### Crop cerra shape 
Crop the data set of cerra (HR) so it fits the low-resolution data set 

In [ ]:
import xarray as xr

# Assuming your list of xarrays is named xarray_list
xarray_list = [...]

# Initialize variables to store max and min longitude values
max_longitude = float('-inf')
min_longitude = float('inf')

max_latitude = float('-inf')
min_latitude = float('inf')

# Iterate through the list of xarrays
for xarray_data in era5:
    # Get the longitude & latitude values from the current xarray
    current_min_lon = xarray_data.longitude.min().values
    current_max_lon = xarray_data.longitude.max().values

    current_min_lat = xarray_data.latitude.min().values
    current_max_lat = xarray_data.latitude.max().values

    min_longitude = min(min_longitude, current_min_lon)
    max_longitude = max(max_longitude, current_max_lon)

    min_latitude = min(min_latitude, current_min_lat)
    max_latitude = max(max_latitude, current_max_lat)

# The variables max_longitude and min_longitude now contain the overall max and min longitude values
print("Max Longitude:", max_longitude)
print("Min Longitude:", min_longitude)
print("")
print("Max Latitude:", max_latitude)
print("Min Latitude:", min_latitude)


Max Longitude: 45.0
Min Longitude: -25.0

Max Latitude: 75.0
Min Latitude: 20.0


In [ ]:
cerra_cropped = []

for xarray in cerra:
    xarray_cropped = xarray.sel(longitude=slice(min_longitude, max_longitude), latitude=slice(min_latitude, max_latitude))
    cerra_cropped.append(xarray_cropped)

display(era5[0].latitude)
display(cerra[0].latitude)
#display(cerra_cropped[0])



<xarray.DataArray 'latitude' (latitude: 221)>
array([75.  , 74.75, 74.5 , ..., 20.5 , 20.25, 20.  ], dtype=float32)
Coordinates:
  * latitude  (latitude) float32 75.0 74.75 74.5 74.25 ... 20.75 20.5 20.25 20.0
Attributes:
    units:      degrees_north
    long_name:  latitude

<xarray.DataArray 'latitude' (latitude: 501)>
array([60.  , 59.95, 59.9 , ..., 35.1 , 35.05, 35.  ], dtype=float32)
Coordinates:
  * latitude  (latitude) float32 60.0 59.95 59.9 59.85 ... 35.15 35.1 35.05 35.0
Attributes:
    units:      degrees_north
    long_name:  latitude

Der Unterschied ist gar nicht, dass es durch longitude & latitude einen größeren Bereich abdeckt. Sondern das mit CERRA einfach sehr viel feinere latitude / longitude abgedeckt ist. Intervall zwischen Data points ist kleiner (era5: 75, 74.75, 74.5,    cerra: 60, 59.95, 59.9)


**Wie gehe ich damit um?**<br>
Ich lasse es so, da es eh gut ist das es feiner ist.<br><br>

**WICHTIG**<br>
was mir dabei aufgefallen ist, der Datenimport ist wohl noch nicht ideal. Besser direkt von EMWCF die Daten verwenden? In der Notiz App habe ich festgehalten wie es am besten funktioniert. Dann kann ich auch gleich das in Arbeitsspeicher laden probieren. <br><br>


**Nächste Steps:**
- Neues Laden probieren und implementieren
- Danach darauf basierend Input Shape finden
- Prüfen wie Input selber für UNet angepasst werden muss (bzw. schon vorhandene Code Teile)
    - Berechnung des Climatology (Nur für Input Channels?)
    - Metrik für Evaluation
    - Model selbst
- Input Data formnen (see questions for chat gpt)

In [ ]:
# Get the shapes from the first xarray in the list
first_era5_xarray = era5[0]
era5_lat_shape = first_era5_xarray['latitude'].shape
era5_lon_shape = first_era5_xarray['longitude'].shape
era5_time_steps = first_era5_xarray['time'].shape[0]

tim = []

# Iterate through the list to find common shapes
for xarr in era5[1:]:
    if xarr['latitude'].shape != era5_lat_shape or xarr['longitude'].shape != era5_lon_shape:
        print("Different latitude/longitude shapes found.")
        
    if xarr['time'].shape[0] != era5_time_steps:
        #print("Different time steps found.")
        tim.append([xarr['time'].shape[0], xarr['time']])
    


print(era5_lat_shape, era5_lon_shape, era5_time_steps)
display(first_era5_xarray)

#for t in tim:
#    print(t[0])
#    print(t[1])


# the month length varies over the year, so the time dimension is not consistent
# simply use the time dimenion of longest month? => so 31 


(221,) (281,) 248


<xarray.Dataset>
Dimensions:    (longitude: 281, latitude: 221, time: 248)
Coordinates:
  * longitude  (longitude) float32 -25.0 -24.75 -24.5 -24.25 ... 44.5 44.75 45.0
  * latitude   (latitude) float32 75.0 74.75 74.5 74.25 ... 20.5 20.25 20.0
  * time       (time) datetime64[ns] 2019-01-01 ... 2019-01-31T21:00:00
Data variables:
    t2m        (time, latitude, longitude) float32 ...
    z          (time, latitude, longitude) float32 1.797e+04 ... 7.262e+03
    lsm        (time, latitude, longitude) float32 1.0 1.0 1.0 ... 1.0 1.0 1.0
Attributes:
    Conventions:  CF-1.6
    history:      2023-05-26 07:39:28 GMT by grib_to_netcdf-2.28.0: grib_to_n...